In [1]:
import scipy.io
import json
import numpy as np
import pathlib
import os

In [2]:
mat = scipy.io.loadmat("C:\\Users\\tiger\\Documents\\GitHub\\FISH-ML\\10_60_min_comparison.mat")

In [78]:
all_cells = mat["Tracked_60min"][0]

In [155]:
def convert(all_cells):
    converted: list[dict()] = []
    for i in range(all_cells.shape[0]):
        cur = dict()
        try:
            for j in range(100):
                cur[j] = all_cells[i][0,0][j][0]
        except IndexError:
            pass
        converted.append(cur)
    return converted

In [165]:
hello = convert(all_cells)
print(hello[0][0])

E:\Hong\12.31.22_Lifetime_FISH\ZprojectWithDAPI\60min_H\
